In [ ]:
### This notebook contains the ML heart of the project.
### After importing the feature data frames a gradient boosting model (XGBoost) is trained and evaluated on an F1 score.

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, log_loss, classification_report
import gc

# Load Feature-Engineered Datasets
print("Loading feature sets...")
try:
    static_features_train_df = pd.read_csv('static_features_train.csv')
    static_features_test_df = pd.read_csv('static_features_test.csv')
    h2h_features_train_df = pd.read_csv('h2h_features_train.csv')
    h2h_features_test_df = pd.read_csv('h2h_features_test.csv')
    rolling_features_train_df = pd.read_csv('rolling_features_train_2_0.csv')
    rolling_features_test_df = pd.read_csv('rolling_features_test_2_0.csv')
    rolling_surface_features_train_df = pd.read_csv('surface_features_train_2_0.csv')
    rolling_surface_features_test_df = pd.read_csv('surface_features_test_2_0.csv')
    fatigue_features_train_df = pd.read_csv('fatigue_features_train.csv')
    fatigue_features_test_df = pd.read_csv('fatigue_features_test.csv')
    elo_features_train_df = pd.read_csv('elo_features_train.csv')
    elo_features_test_df = pd.read_csv('elo_features_test.csv')

    print("✅ Feature sets loaded successfully.")
except FileNotFoundError as e:
    print(f"Error: Could not find a required file: {e.filename}")
    exit()

train_df = pd.concat([static_features_train_df, h2h_features_train_df, rolling_features_train_df, rolling_surface_features_train_df, fatigue_features_train_df, elo_features_train_df],axis=1)
test_df = pd.concat([static_features_test_df, h2h_features_test_df, rolling_features_test_df, rolling_surface_features_test_df, fatigue_features_test_df, elo_features_test_df], axis=1)

pd.set_option('display.max_columns', None)

# Clean up memory
del static_features_train_df, static_features_test_df, h2h_features_train_df, h2h_features_test_df, rolling_features_train_df, rolling_features_test_df, rolling_surface_features_train_df, rolling_surface_features_test_df, fatigue_features_train_df, fatigue_features_test_df, elo_features_train_df, elo_features_test_df
gc.collect()


Loading feature sets...
✅ Feature sets loaded successfully.


,p1_rank,p2_rank,rank_diff,p1_age,p2_age,age_diff,p1_height,p2_height,height_diff,target,surface_Clay,surface_Grass,surface_Hard,round_BR,round_F,round_QF,round_R128,round_R16,round_R32,round_R64,round_RR,round_SF,p1_h2h_wins,p2_h2h_wins,h2h_matches_played,p1_h2h_win_pc,p2_h2h_win_pc,diff_h2h_win_pc,p1_h2h_wins_Clay,p2_h2h_wins_Clay,h2h_matches_played_Clay,p1_h2h_win_pc_Clay,p2_h2h_win_pc_Clay,diff_h2h_win_pc_Clay,p1_h2h_wins_Grass,p2_h2h_wins_Grass,h2h_matches_played_Grass,p1_h2h_win_pc_Grass,p2_h2h_win_pc_Grass,diff_h2h_win_pc_Grass,p1_h2h_wins_Hard,p2_h2h_wins_Hard,h2h_matches_played_Hard,p1_h2h_win_pc_Hard,p2_h2h_win_pc_Hard,diff_h2h_win_pc_Hard,p1_win_pc_90d,p1_matches_played_90d,p1_ace_ratio_90d,p1_df_ratio_90d,p1_ace_vs_df_ratio_90d,p1_1st_serve_in_pc_90d,p1_1st_serve_win_pc_90d,p1_2nd_serve_win_pc_90d,p1_return_win_pc_90d,p1_bp_save_pc_90d,p1_bp_conversion_pc_90d,p1_tiebreak_win_pc_90d,p1_win_pc_vs_top10_90d,p2_win_pc_90d,p2_matches_played_90d,p2_ace_ratio_90d,p2_df_ratio_90d,p2_ace_vs_df_ratio_90d,p2_1st_serve_in_pc_90d,p2_1st_serve_win_pc_90d,p2_2nd_serve_win_pc_90d,p2_return_win_pc_90d,p2_bp_save_pc_90d,p2_bp_conversion_pc_90d,p2_tiebreak_win_pc_90d,p2_win_pc_vs_top10_90d,diff_win_pc_90d,diff_matches_played_90d,diff_ace_ratio_90d,diff_df_ratio_90d,diff_ace_vs_df_ratio_90d,diff_1st_serve_in_pc_90d,diff_1st_serve_win_pc_90d,diff_2nd_serve_win_pc_90d,diff_return_win_pc_90d,diff_bp_save_pc_90d,diff_bp_conversion_pc_90d,diff_tiebreak_win_pc_90d,diff_win_pc_vs_top10_90d,p1_win_pc_180d,p1_matches_played_180d,p1_ace_ratio_180d,p1_df_ratio_180d,p1_ace_vs_df_ratio_180d,p1_1st_serve_in_pc_180d,p1_1st_serve_win_pc_180d,p1_2nd_serve_win_pc_180d,p1_return_win_pc_180d,p1_bp_save_pc_180d,p1_bp_conversion_pc_180d,p1_tiebreak_win_pc_180d,p1_win_pc_vs_top10_180d,p2_win_pc_180d,p2_matches_played_180d,p2_ace_ratio_180d,p2_df_ratio_180d,p2_ace_vs_df_ratio_180d,p2_1st_serve_in_pc_180d,p2_1st_serve_win_pc_180d,p2_2nd_serve_win_pc_180d,p2_return_win_pc_180d,p2_bp_save_pc_180d,p2_bp_conversion_pc_180d,p2_tiebreak_win_pc_180d,p2_win_pc_vs_top10_180d,diff_win_pc_180d,diff_matches_played_180d,diff_ace_ratio_180d,diff_df_ratio_180d,diff_ace_vs_df_ratio_180d,diff_1st_serve_in_pc_180d,diff_1st_serve_win_pc_180d,diff_2nd_serve_win_pc_180d,diff_return_win_pc_180d,diff_bp_save_pc_180d,diff_bp_conversion_pc_180d,diff_tiebreak_win_pc_180d,diff_win_pc_vs_top10_180d,p1_win_pc_360d,p1_matches_played_360d,p1_ace_ratio_360d,p1_df_ratio_360d,p1_ace_vs_df_ratio_360d,p1_1st_serve_in_pc_360d,p1_1st_serve_win_pc_360d,p1_2nd_serve_win_pc_360d,p1_return_win_pc_360d,p1_bp_save_pc_360d,p1_bp_conversion_pc_360d,p1_tiebreak_win_pc_360d,p1_win_pc_vs_top10_360d,p2_win_pc_360d,p2_matches_played_360d,p2_ace_ratio_360d,p2_df_ratio_360d,p2_ace_vs_df_ratio_360d,p2_1st_serve_in_pc_360d,p2_1st_serve_win_pc_360d,p2_2nd_serve_win_pc_360d,p2_return_win_pc_360d,p2_bp_save_pc_360d,p2_bp_conversion_pc_360d,p2_tiebreak_win_pc_360d,p2_win_pc_vs_top10_360d,diff_win_pc_360d,diff_matches_played_360d,diff_ace_ratio_360d,diff_df_ratio_360d,diff_ace_vs_df_ratio_360d,diff_1st_serve_in_pc_360d,diff_1st_serve_win_pc_360d,diff_2nd_serve_win_pc_360d,diff_return_win_pc_360d,diff_bp_save_pc_360d,diff_bp_conversion_pc_360d,diff_tiebreak_win_pc_360d,diff_win_pc_vs_top10_360d,p1_win_pc_Hard_90d,p1_matches_played_Hard_90d,p1_ace_ratio_Hard_90d,p1_df_ratio_Hard_90d,p1_ace_vs_df_ratio_Hard_90d,p1_1st_serve_in_pc_Hard_90d,p1_1st_serve_win_pc_Hard_90d,p1_2nd_serve_win_pc_Hard_90d,p1_return_win_pc_Hard_90d,p1_bp_save_pc_Hard_90d,p1_bp_conversion_pc_Hard_90d,p1_tiebreak_win_pc_Hard_90d,p1_win_pc_vs_top10_Hard_90d,p2_win_pc_Hard_90d,p2_matches_played_Hard_90d,p2_ace_ratio_Hard_90d,p2_df_ratio_Hard_90d,p2_ace_vs_df_ratio_Hard_90d,p2_1st_serve_in_pc_Hard_90d,p2_1st_serve_win_pc_Hard_90d,p2_2nd_serve_win_pc_Hard_90d,p2_return_win_pc_Hard_90d,p2_bp_save_pc_Hard_90d,p2_bp_conversion_pc_Hard_90d,p2_tiebreak_win_pc_Hard_90d,p2_win_pc_vs_top10_Hard_90d,diff_win_pc_Hard_90d,diff_matches_played_Hard_90d,diff_ace_ratio

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59987 entries, 0 to 59986
Columns: 547 entries, p1_rank to diff_ELO_hard
dtypes: bool(12), float64(510), int64(25)
memory usage: 245.5 MB


None

0

In [ ]:
### The following two notebooks set the baseline and demonstrate how predictive stats like rank or ELO are.

In [ ]:
### Get baseline F1 score for the simple prediction rule 'The higher rated player wins.'

baseline_prediction = np.where(test_df['p1_rank'] < test_df['p2_rank'], 1, 0)
actual_outcomes = test_df['target']

# Get overall accuracy
baseline_accuracy = accuracy_score(actual_outcomes, baseline_prediction)
print(f"Baseline Accuracy: {baseline_accuracy * 100:.2f}%")

# Generate report to get F1 score
print("\n--- Baseline Classification Report ---")
print(classification_report(actual_outcomes, baseline_prediction, target_names=['P2 Wins', 'P1 Wins']))

# Also write a function which gives the classification report on the expected/upset cases

def classification_report_on_expected_or_upset_outcomes(test_df,actual_outcomes, predictions):
    # An upset occurs if the higher-ranked player (lower rank number) lost.
    # We need to determine who the higher-ranked player was in each row.
    is_p1_higher_ranked = test_df['p1_rank'] < test_df['p2_rank']

    # An upset happened if:
    #   - P1 was higher ranked BUT lost (target is 0)
    #   - P2 was higher ranked BUT lost (target is 1)
    upset_mask = (is_p1_higher_ranked & (test_df['target'] == 0)) | \
                (~is_p1_higher_ranked & (test_df['target'] == 1))

    # Expected outcomes are all other matches
    expected_mask = ~upset_mask

    # Filter the true outcomes and the model's predictions for each group
    y_test_expected = actual_outcomes[expected_mask]
    preds_expected = predictions[expected_mask]

    y_test_upsets = actual_outcomes[upset_mask]
    preds_upsets = predictions[upset_mask]

    print(f"\nFound {len(y_test_expected)} expected outcomes in the test set.")
    print(f"Found {len(y_test_upsets)} upsets in the test set.")


    # Generate a separate classification report for each group

    print("\n--- Performance on EXPECTED OUTCOMES ---")
    # Note: For this subset, the "correct" prediction is that the favorite wins.
    print(classification_report(y_test_expected, preds_expected))


    print("\n--- Performance on UPSETS ---")
    print(classification_report(y_test_upsets, preds_upsets))



Baseline Accuracy: 64.02%

--- Baseline Classification Report ---
              precision    recall  f1-score   support

     P2 Wins       0.64      0.61      0.62      2380
     P1 Wins       0.64      0.67      0.66      2514

    accuracy                           0.64      4894
   macro avg       0.64      0.64      0.64      4894
weighted avg       0.64      0.64      0.64      4894



In [4]:
# ELO baseline
from sklearn.metrics import confusion_matrix

pred_rule = (test_df['diff_ELO'] > 0).astype(int)
acc_rule = accuracy_score(test_df['target'], pred_rule)
print('Deterministic rule accuracy (p1 wins if p1_ELO>p2_ELO):', acc_rule)
print('Confusion matrix (p1_won vs pred):\n', confusion_matrix(test_df['target'], pred_rule))

Deterministic rule accuracy (p1 wins if p1_ELO>p2_ELO): 0.6360850020433183
Confusion matrix (p1_won vs pred):
 [[1490  890]
 [ 891 1623]]


In [ ]:
### The below cell is the "control room" with which to choose which features to include in the model training.
### This enables us to experiment quickly which features yield good results in the model training and which are redundant.

In [ ]:
### Experiment with different feature sets

FEATURE_CONFIG = {
    # --- Switches for each feature family ---
    'use_static_features': True,
    'use_h2h_features': True,
    'use_fatigue_features': True,
    'use_elo_features': True,
    'use_rolling_features': True, # Master switch for all rolling form stats

    # --- Fine-grained settings for Rolling Features ---
    # This allows you to set different time windows for overall vs. surface stats
    'rolling_overall_windows': [90, 180, 360], # e.g., [90, 360]
    'rolling_surface_windows': [90, 180, 360],      # e.g., [90] or [] to disable
    'rolling_surface_surfaces': ['Clay', 'Grass', 'Hard'], #'Clay', 'Grass', 'Hard'

    # Switches for the different groups within rolling features
    'rolling_use_serve_stats': True,
    'rolling_use_return_stats': True,
    'rolling_use_pressure_stats': True,
    'rolling_use_outcome_stats': True
}


# --- 2. Define the Blueprints ---
# A. Map conceptual group names to their feature files
feature_files = {
    'static': {'train': 'static_features_train.csv', 'test': 'static_features_test.csv'},
    'h2h': {'train': 'h2h_features_train.csv', 'test': 'h2h_features_test.csv'},
    'fatigue': {'train': 'fatigue_features_train.csv', 'test': 'fatigue_features_test.csv'},
    'elo': {'train': 'elo_features_train.csv', 'test': 'elo_features_test.csv'},
    'rolling_overall': {'train': 'rolling_features_train_2_0.csv', 'test': 'rolling_features_test_2_0.csv'},
    'rolling_surface': {'train': 'surface_features_train_2_0.csv', 'test': 'surface_features_test_2_0.csv'}
}

# B. Map rolling group names to their base stat names
rolling_groups = {
    'serve_stats': ['ace_ratio', 'df_ratio', '1st_serve_in_pc', '1st_serve_win_pc', '2nd_serve_win_pc', 'ace_vs_df_ratio'],
    'return_stats': ['return_win_pc'],
    'pressure_stats': ['bp_save_pc', 'bp_conversion_pc', 'tiebreak_win_pc'],
    'outcome_stats': ['win_pc', 'win_pc_vs_top10', 'matches_played']
}


# --- 3. The Intelligent Feature Loader & Builder ---
active_train_dfs = []
active_test_dfs = []
print("Building feature set based on config:")

# A. Load the simple feature sets based on switches
simple_feature_sets = ['static', 'h2h', 'fatigue', 'elo']
for feature_name in simple_feature_sets:
    if FEATURE_CONFIG.get(f'use_{feature_name}_features'):
        print(f"- Including: {feature_name.capitalize()} Features")
        active_train_dfs.append(pd.read_csv(feature_files[feature_name]['train']))
        active_test_dfs.append(pd.read_csv(feature_files[feature_name]['test']))

# B. Dynamically build and filter the rolling feature sets
if FEATURE_CONFIG['use_rolling_features']:
    print("- Including: Rolling Features with specified settings")
    # Load the raw rolling feature files
    rolling_train_df = pd.read_csv(feature_files['rolling_overall']['train'])
    rolling_test_df = pd.read_csv(feature_files['rolling_overall']['test'])
    surface_train_df = pd.read_csv(feature_files['rolling_surface']['train'])
    surface_test_df = pd.read_csv(feature_files['rolling_surface']['test'])

    cols_to_keep = []
    # Loop through the defined groups
    for group_name, base_stats in rolling_groups.items():
        if FEATURE_CONFIG.get(f'rolling_use_{group_name}'):
            for stat in base_stats:
                # Add columns for the selected overall time windows
                for window in FEATURE_CONFIG['rolling_overall_windows']:
                    cols_to_keep.extend([f'p1_{stat}_{window}d', f'p2_{stat}_{window}d', f'diff_{stat}_{window}d'])
                # Add columns for the selected surface time windows
                for window in FEATURE_CONFIG['rolling_surface_windows']:
                    # Note the different naming convention for surface stats
                    for surface in FEATURE_CONFIG['rolling_surface_surfaces']:
                        cols_to_keep.extend([f'p1_{stat}_{surface}_{window}d', f'p2_{stat}_{surface}_{window}d', f'diff_{stat}_{surface}_{window}d'])

    # Keep only the selected columns from the loaded DataFrames
    active_train_dfs.append(rolling_train_df[[col for col in cols_to_keep if col in rolling_train_df.columns]])
    active_test_dfs.append(rolling_test_df[[col for col in cols_to_keep if col in rolling_test_df.columns]])
    active_train_dfs.append(surface_train_df[[col for col in cols_to_keep if col in surface_train_df.columns]])
    active_test_dfs.append(surface_test_df[[col for col in cols_to_keep if col in surface_test_df.columns]])


final_features_df_train = pd.concat(active_train_dfs, axis=1)
final_features_df_test = pd.concat(active_test_dfs, axis=1)


Building feature set based on config:
- Including: Static Features
- Including: H2h Features
- Including: Fatigue Features
- Including: Elo Features
- Including: Rolling Features with specified settings
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59987 entries, 0 to 59986
Columns: 547 entries, p1_rank to diff_matches_played_Hard_360d
dtypes: bool(12), float64(510), int64(25)
memory usage: 245.5 MB


None

In [ ]:
### Another option to control the model training is balancing the training set. 
### Since the win of the favourite (i.e. higher ranked) is to be expected, they statistically occur more often. This biases the model to 
### prefer the favourite over the underdog more often then it should. Artifically downsampling is a way of controlling this bias but the technique comes of course
### with a tradeoff. Downsample too much and the model will be overconfident in the underdog. Tweaking of this parameter will be performed in another notebook.

In [37]:
### use this to balance dataset if wanted, i.e. downsample expected outcomes


def filter_df_into_upset_classes(df):
    is_p1_higher_ranked = df['p1_rank'] < df['p2_rank']

    # An upset happened if:
    #   - P1 was higher ranked BUT lost (target is 0)
    #   - P2 was higher ranked BUT lost (target is 1)
    upset_mask = (is_p1_higher_ranked & (df['target'] == 0)) | \
                (~is_p1_higher_ranked & (df['target'] == 1))

    # Expected outcomes are all other matches
    expected_mask = ~upset_mask

    return df[expected_mask].copy(), df[upset_mask].copy()

final_features_df_train_expected, final_features_df_train_upset = filter_df_into_upset_classes(final_features_df_train)

n_expected = len(final_features_df_train_expected)
n_upset = len(final_features_df_train_upset)

upset_ratio = n_upset/n_expected

print(f'The number of expected outcomes in the training set is {n_expected} and the number of upsets is {n_upset}.')
print(f'Their ratio is {upset_ratio}.')

desired_ratio = 1
print(f'The desired ratio is {desired_ratio}.')

n_expected_downsampled = int((1/desired_ratio) * n_upset)
new_starting_index = int(n_expected - n_expected_downsampled)

print(f'The used number of expected outcomes is now {n_expected_downsampled}.')

final_features_df_train_expected = final_features_df_train_expected.iloc[new_starting_index:38920].copy()

final_features_df_train_balanced = pd.concat([final_features_df_train_expected, final_features_df_train_upset], axis = 0)


The number of expected outcomes in the training set is 38920 and the number of upsets is 21067.
Their ratio is 0.541289825282631.
The desired ratio is 1.
The used number of expected outcomes is now 21067.


In [ ]:
### MAIN TRAINING CELL with F1 score evaluation

In [38]:
### Training and evaluating the XGBoost Model

#  Define Training and Testing Sets 
# The target is the 'target' column we created
X_train = final_features_df_train_balanced.drop(['target'], axis=1)
y_train = final_features_df_train_balanced['target']

X_test = final_features_df_test.drop(['target'], axis=1)
y_test = final_features_df_test['target']

print(f"\n✅ Final feature set created with {X_train.shape[1]} features.")

print(f"\nTraining on {len(X_train)} match instances.")
print(f"Testing on {len(X_test)} match instances.")

#  Train the XGBoost Model 
print("\nTraining the XGBoost model...")
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_estimators=1000, 
    learning_rate=0.03,
    max_depth=2,
    subsample=0.7,
    colsample_bytree=0.7
)

# Fit the model to our training data
model.fit(X_train, y_train, verbose=False)
print("✅ Model training complete!")

# Evaluate the Model on the Unseen Test Set
print("\nEvaluating model performance...")
predictions = model.predict(X_test)
pred_probs = model.predict_proba(X_test)[:, 1]

# Calculate key performance metrics
accuracy = accuracy_score(y_test, predictions)
logloss = log_loss(y_test, pred_probs)

print("\n--- Model Evaluation Results on Test Data (2024-2025) ---")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Log Loss: {logloss:.4f} (Lower is better)")
print("\n--- Classification Report ---")
print(classification_report(y_test, predictions))

# Feature Importance 
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n--- Top Most Important Features ---")
display(feature_importance.head(15))

classification_report_on_expected_or_upset_outcomes(final_features_df_test, y_test, predictions)


✅ Final feature set created with 546 features.

Training on 42134 match instances.
Testing on 4894 match instances.

Training the XGBoost model...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:36:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Model training complete!

Evaluating model performance...

--- Model Evaluation Results on Test Data (2024-2025) ---
Accuracy: 60.01%
Log Loss: 0.6489 (Lower is better)

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.59      0.57      0.58      2380
           1       0.61      0.63      0.62      2514

    accuracy                           0.60      4894
   macro avg       0.60      0.60      0.60      4894
weighted avg       0.60      0.60      0.60      4894


--- Top Most Important Features ---


,feature,importance
77,diff_ELO_hard,0.019726
68,diff_ELO,0.019631
107,diff_1st_serve_win_pc_90d,0.010880
268,p2_1st_serve_in_pc_Clay_360d,0.010318
59,diff_matches_last_30d,0.008441
2,rank_diff,0.008224
103,p2_1st_serve_in_pc_360d,0.008054
110,diff_1st_serve_win_pc_180d,0.007534
353,diff_ace_vs_df_ratio_Grass_360d,0.007006
74,diff_ELO_grass,0.006950



Found 3133 expected outcomes in the test set.
Found 1761 upsets in the test set.

--- Performance on EXPECTED OUTCOMES ---
              precision    recall  f1-score   support

           0       0.53      0.53      0.53      1440
           1       0.60      0.60      0.60      1693

    accuracy                           0.57      3133
   macro avg       0.56      0.56      0.56      3133
weighted avg       0.57      0.57      0.57      3133


--- Performance on UPSETS ---
              precision    recall  f1-score   support

           0       0.70      0.63      0.67       940
           1       0.62      0.70      0.66       821

    accuracy                           0.66      1761
   macro avg       0.66      0.66      0.66      1761
weighted avg       0.67      0.66      0.66      1761



In [ ]:
### Importing the model parameters to use in other notebooks.

In [39]:
import joblib

# (This comes after your model.fit(X_train, y_train) line)

# Define a filename for your model
model_filename = 'tennis_predictor_model_balanced_1_0.joblib'

# Save the trained model object to the file
joblib.dump(model, model_filename)

print(f"\n✅ Model successfully saved to {model_filename}")


✅ Model successfully saved to tennis_predictor_model_balanced_1_0.joblib
